<h2>Empirical Tests for Non Stationary Data</h2>
<h3>By Beatriz Ottoboni</h3>

Programa de Pós Graduação Ciência da Computação<br>
Disciplina Inteligência Artificial<br>
Professor Didier Oliveros

<h2>Summary</h2>
<h3><a href='#dataset'>1. Dataset Description</a></h3>
<h3><a href='#stat_models'>2. Statistical Models</a></h3>
<h4><a href='#arma'>2.1. ARMA</a><br>
<a href='#sarma'>2.2. Seasonal ARMA</a><br>
<a href='#sarima'>2.3. SARIMA</a><br>
<a href='#prophet'>2.4. Prophet</a></h4>
<h3><a href='#nn_models'>3. Neural Networks Models</a></h3>
<a href='#rnn'><h4>3.1. RNN</a><br>
<a href='#rnn_emd'>3.2. RNN with CEEMDAN</a><br>
<a href='#lstm'>3.3. LSTM</a><br>
<a href='#lstm_emd'>3.3. LSTM with CEEMDAN</a><br>
<a href='#n_prophet'>3.4. NeuralProphet</a><br>
<a href='#transformer'>3.5. Transformer</a><br>
<a href='#transformer_emd'>3.6. Transformer with CEEMDAN</a></h4>

<a id='dataset'></a><h2>1. Dataset Description

<h4>Libraries Versions:</h4>
pandas==2.2.2<br>
numpy==1.26.4 (statistical models) / numpy==2.0.2 (neural network models)<br>
pmdarima==2.0.4<br>
statsmodels==0.14.5<br>
prophet==1.1.7<br>
sklearn==1.6.1<br>
math==1.1.7<br>
PyEMD==1.0.0<br>
EMD-signal==1.6.4<br>
scikeras==0.13.0<br>
neuralprophet==0.9.0<br>
matplotlib==3.10.0<br>
tsfm_public==0.3.2.dev16+g63c962f<br>
transformers==4.53.1<br>
tensorflow==2.18.0<br>
torch==2.6.0+cu124<br>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import statsmodels
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

import pmdarima as pm
import prophet

import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from math import sqrt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = ''

In [ ]:
df = pd.read_excel(f'{path}/ipeadata_mensal.xlsx', sheet_name='consumo')
df = df[['Data', 'Consumo']]
df.columns = ['date', 'energy_consumption']
df = df.set_index('date')

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['energy_consumption'])
plt.title('Energy Consumption over Time')
plt.xlabel('date')
plt.ylabel('consumption')
plt.grid(True)
plt.show()

Non Stationarity Test

In [ ]:
print("ADF Test")
adf_result = adfuller(df)
print(f'ADF Statistic: {adf_result[0]}')
print(f'p-value: {adf_result[1]}')
print("Non stationary (p-value > 0.1)") if adf_result[1] > 0.1 else print("Stationary (p-value <= 0.1)")

In [ ]:
fig, ax1 = plt.subplots( figsize=(10, 6))

plot_acf(df, lags=24, ax=ax1)
ax1.set_title('Autocorrelation Function (ACF)', fontdict={'fontsize': 25})

plt.show()

In [ ]:
fig, ax2 = plt.subplots(1, figsize=(10, 6))

plot_pacf(df, lags=24, ax=ax2, method='ols')
ax2.set_title('Partial Autocorrelation Function (PACF)', fontdict={'fontsize': 25})

<a id='stat_models'></a><h2>2. Statistical Models

In [ ]:
df = df.dropna()

df['energy_consumption_shift'] = df['energy_consumption'].shift(-1)

X = df.values
size = int(len(X) * 0.9)
train, test = df.iloc[0:size], df.iloc[size:len(X)]

scaler = MinMaxScaler(feature_range=(0, 1))
train['energy_c_scaler'] = scaler.fit_transform(train[['energy_consumption']])
test['energy_c_scaler'] = scaler.transform(test[['energy_consumption']])
df['energy_c_scaler'] = scaler.transform(df[['energy_consumption']])

train['energy_c_scaler_shift'] = train['energy_c_scaler'].shift(-1)
test['energy_c_scaler_shift'] = test['energy_c_scaler'].shift(-1)
df['energy_c_scaler_shift'] = df['energy_c_scaler'].shift(-1)

test = test.dropna()
y_test = test['energy_consumption'].values

<a id='arma'></a><h3>2.1. ARMA

In [ ]:
model_arma_selection = pm.auto_arima(train['energy_c_scaler'],
                      start_p=0, max_p=3,
                      max_d=0,
                      start_q=0, max_q=12,
                      seasonal=False,
                      trace=True,
                      stationary=True,
                      n_fits=10,
                      error_action='ignore',
                      scoring='mse',
                      suppress_warnings=True,
                      stepwise=True)


In [ ]:
history = train.dropna()
test_arma = test.copy()
predictions_arma = list()

# walk-forward validation
for t in range(len(test)):
  model_arma = pm.arima.ARIMA(order=model_arma_selection.order, seasonal_order=model_arma_selection.seasonal_order, maxiter=50, scoring='mse')
  model_arma.fit(history['energy_c_scaler'])
  y_pred_arma = model_arma.predict().iloc[:1].values
  predictions_arma.append(y_pred_arma)
  history = pd.concat([history, test_arma.iloc[:1]])
  test_arma = test_arma.iloc[1:]

In [ ]:
predictions_arma = np.array(predictions_arma).reshape(-1, 1)
predictions_arma_transform = scaler.inverse_transform(predictions_arma).reshape(-1)

# evaluate forecasts
rmse = sqrt(mean_squared_error(y_test, predictions_arma_transform.reshape(-1)))
mape = mean_absolute_percentage_error(y_test, predictions_arma_transform.reshape(-1))
mae = np.abs(y_test - predictions_arma_transform.reshape(-1)).mean()
results_arma = pd.DataFrame(data={'model': f'ARMA {model_arma.order} {model_arma.seasonal_order}', 'rmse':[rmse], 'mape':[mape], 'mae':[mae]})

In [ ]:
pd.DataFrame(predictions_arma_transform).to_csv(f'{path}/preds_arma.csv', index=False)

In [ ]:
results_arma

<a id='sarma'></a><h3>2.2. Seasonal ARMA

In [ ]:
model_s_arma_selection = pm.auto_arima(train['energy_c_scaler'],
                      start_p=0, max_p=3,
                      max_d=0,
                      start_q=0, max_q=6,
                      start_P=0, max_P=3,
                      max_D=0,
                      start_Q=0, max_Q=6,
                      max_order=12,
                      m=12, seasonal=True,
                      trace=True,
                      stationary=True,
                      n_fits=10,
                      error_action='ignore',
                      scoring='mse',
                      suppress_warnings=True,
                      stepwise=True)


In [ ]:
history = train.dropna()
test_s_arma = test.copy()
predictions_s_arma = list()

# walk-forward validation
for t in range(len(test)):
  model_s_arma = pm.arima.ARIMA(order=model_s_arma_selection.order, seasonal_order=model_s_arma_selection.seasonal_order, maxiter=50, scoring='mse')
  model_s_arma.fit(history['energy_c_scaler'])
  y_pred_s_arma = model_s_arma.predict().iloc[:1].values
  predictions_s_arma.append(y_pred_s_arma)
  history = pd.concat([history, test_s_arma.iloc[:1]])
  test_s_arma = test_s_arma.iloc[1:]



In [ ]:
predictions_s_arma = np.array(predictions_s_arma).reshape(-1, 1)
predictions_s_arma_transform = scaler.inverse_transform(predictions_s_arma).reshape(-1)

# evaluate forecasts
rmse = sqrt(mean_squared_error(y_test, predictions_s_arma_transform.reshape(-1)))
mape = mean_absolute_percentage_error(y_test, predictions_s_arma_transform.reshape(-1))
mae = np.abs(y_test - predictions_s_arma_transform.reshape(-1)).mean()
results_s_arma = pd.DataFrame(data={'model': f'Seasonal ARMA {model_s_arma.order} {model_s_arma.seasonal_order}', 'rmse':[rmse], 'mape':[mape], 'mae':[mae]})

In [ ]:
pd.DataFrame(predictions_s_arma_transform).to_csv(f'{path}/preds_s_arma.csv', index=False)

In [ ]:
results_s_arma

<a id='sarima'></a><h3>2.3. SARIMA

In [ ]:
model_sarima_selection = pm.auto_arima(train['energy_c_scaler'],
                      start_p=0, max_p=3,
                      max_d=6,
                      start_q=0, max_q=12,
                      start_P=0, max_P=3,
                      max_D=6,
                      start_Q=0, max_Q=12,
                      max_order=12,
                      m=12, seasonal=True,
                      trace=True,
                      stationary=False,
                      n_fits=10,
                      error_action='ignore',
                      scoring='mse',
                      suppress_warnings=True,
                      stepwise=True)


In [ ]:
history = train.dropna()
test_sarima = test.copy()
predictions_sarima = list()

# walk-forward validation
for t in range(len(test)):
  model_sarima = pm.arima.ARIMA(order=model_sarima_selection.order, seasonal_order=model_sarima_selection.seasonal_order, maxiter=50, scoring='mse')
  model_sarima.fit(history['energy_c_scaler'])
  y_pred_sarima = model_sarima.predict().iloc[:1].values
  predictions_sarima.append(y_pred_sarima)
  history = pd.concat([history, test_sarima.iloc[:1]])
  test_sarima = test_sarima.iloc[1:]

In [ ]:
predictions_sarima = np.array(predictions_sarima).reshape(-1, 1)
predictions_sarima_transform = scaler.inverse_transform(predictions_sarima).reshape(-1)

# evaluate forecasts
rmse = sqrt(mean_squared_error(y_test, predictions_sarima_transform.reshape(-1)))
mape = mean_absolute_percentage_error(y_test, predictions_sarima_transform.reshape(-1))
mae = np.abs(y_test - predictions_sarima_transform.reshape(-1)).mean()
results_sarima = pd.DataFrame(data={'model': f'SARIMA {model_sarima.order} {model_sarima.seasonal_order}', 'rmse':[rmse], 'mape':[mape], 'mae':[mae]})

In [ ]:
pd.DataFrame(predictions_sarima_transform).to_csv(f'{path}/preds_sarima.csv', index=False)

In [ ]:
results_sarima

<a id='prophet'></a><h3>2.4. Prophet

In [ ]:
train_prophet = train.reset_index()
train_prophet = train_prophet[['date', 'energy_c_scaler']]
train_prophet.columns = ['ds', 'y']

test_prophet = test.reset_index()
test_prophet = test_prophet[['date', 'energy_c_scaler']]
test_prophet.columns = ['ds', 'y']

In [ ]:
history = train_prophet.dropna()
test_prophet_val = test_prophet.copy()
predictions_prophet = pd.DataFrame()

# walk-forward validation
for t in range(len(test_prophet_val)):
  model_prophet = prophet.Prophet()
  model_prophet.fit(history)
  y_pred_prophet = model_prophet.predict(test_prophet_val.iloc[:1][['ds']])[['ds', 'yhat']]
  predictions_prophet = pd.concat([predictions_prophet, y_pred_prophet])
  history = pd.concat([history, test_prophet_val.iloc[:1]])
  test_prophet_val = test_prophet_val.iloc[1:]

In [ ]:
predictions_prophet_transform = np.array(predictions_prophet['yhat']).reshape(-1, 1)
predictions_prophet_transform = scaler.inverse_transform(predictions_prophet_transform).reshape(-1)

# evaluate forecasts
rmse = sqrt(mean_squared_error(y_test, predictions_prophet_transform))
mape = mean_absolute_percentage_error(y_test, predictions_prophet_transform)
mae = np.abs(y_test - predictions_prophet_transform).mean()

print(f'Test RMSE: {rmse}')
print(f'Test MAPE: {mape}')
print(f'Test MAE: {mae}')

# Store the results
results_prophet = pd.DataFrame(data={'model': f"Prophet", 'rmse':[rmse], 'mape':[mape], 'mae':[mae]})

In [ ]:
pd.DataFrame(predictions_prophet_transform).to_csv(f'{path}/preds_prophet.csv', index=False)

In [ ]:
results_prophet

<a id='nn_models'></a><h2>3. Neural Networks Models

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PyEMD.CEEMDAN import CEEMDAN
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from math import sqrt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN
from neuralprophet import NeuralProphet

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

from transformers import (
    EarlyStoppingCallback,
    PatchTSTConfig,
    PatchTSTForPrediction,
    set_seed,
    Trainer,
    TrainingArguments
)

from tsfm_public.toolkit.dataset import ForecastDFDataset
from tsfm_public.toolkit.time_series_preprocessor import TimeSeriesPreprocessor
from tsfm_public.toolkit.util import select_by_index

warnings.filterwarnings("ignore", module="torch")

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
df = pd.read_excel(f'{path}/ipeadata_mensal.xlsx', sheet_name='consumo')
df = df[['Data', 'Consumo']]
df.columns = ['date', 'energy_consumption']
df['energy_consumption_shift'] = df['energy_consumption'].shift(-1)

In [ ]:
X = df.values
size = int(len(X) * 0.9)
train, test = df.iloc[0:size], df.iloc[size:len(X)]

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
train['energy_c_scaler'] = scaler.fit_transform(train[['energy_consumption']])
test['energy_c_scaler'] = scaler.transform(test[['energy_consumption']])
df['energy_c_scaler'] = scaler.transform(df[['energy_consumption']])


train['energy_c_scaler_shift'] = train['energy_c_scaler'].shift(-1)
test['energy_c_scaler_shift'] = test['energy_c_scaler'].shift(-1)
df['energy_c_scaler_shift'] = df['energy_c_scaler'].shift(-1)

test = test.dropna()
y_test = test['energy_consumption_shift'].values

In [ ]:
num_imf = 4
ceemdan = CEEMDAN()

IMFs = pd.DataFrame(ceemdan(np.array(train['energy_c_scaler']), max_imf=num_imf)).T
train_imf = pd.concat([train, IMFs], axis=1)
train_imf.columns = ['date', 'energy_consumption', 'energy_consumption_shift', 'energy_c_scaler', 'energy_c_scaler_shift', 'imf_0', 'imf_1', 'imf_2', 'imf_3', 'imf_4']
fig, axes = plt.subplots(num_imf+1, 1, figsize=(12, 1 * (num_imf+1)), sharex=True)
for i in range(num_imf+1):
  axes[i].plot(IMFs[i], label=f'IMF {i}')
  axes[i].legend(loc='upper right')
  axes[i].grid(True)
  axes[-1].set_xlabel('Amostras')
  plt.tight_layout()
plt.show()

In [ ]:
num_imf = 4
ceemdan = CEEMDAN()
IMFs_total = pd.DataFrame(ceemdan(np.array(df['energy_c_scaler']), max_imf=num_imf)).T
imf = pd.concat([df, IMFs_total], axis=1)
imf.columns = ['date', 'energy_consumption', 'energy_consumption_shift', 'energy_c_scaler', 'energy_c_scaler_shift', 'imf_0', 'imf_1', 'imf_2', 'imf_3', 'imf_4']

In [ ]:
train_imf = train_imf.dropna()
imf = imf.dropna()

<a id='rnn'></a><h3>3.1. RNN

In [ ]:
history = [x for x in train]
predictions_rnn = list()

train_rnn = train_imf[['energy_c_scaler','energy_c_scaler_shift']]
test_rnn = imf.iloc[size:-1][['energy_c_scaler', 'energy_c_scaler_shift']]
print(train_rnn.shape, test_rnn.shape)

# walk-forward validation
for t in range(len(test)-1):

  X_train_rnn = train_rnn[['energy_c_scaler']]
  y_train_rnn = train_rnn['energy_c_scaler_shift']

  X_test_rnn = test_rnn[['energy_c_scaler']]
  y_test_rnn = test_rnn['energy_c_scaler_shift']

  size_train = int(len(X_train_rnn.values) * 0.9)
  X_val_rnn = X_train_rnn[size_train:]
  y_val_rnn = X_train_rnn[size_train:]
  X_train_rnn_ = X_train_rnn.iloc[0:size_train]
  y_train_rnn_ = y_train_rnn.iloc[0:size_train]

  X_test_reshape = np.reshape(X_test_rnn, (X_test_rnn.shape[0], 1, X_test_rnn.shape[1]))
  y_test_reshape = np.array(y_test_rnn)

  X_train_reshape = np.reshape(X_train_rnn_, (X_train_rnn_.shape[0], 1, X_train_rnn_.shape[1]))
  y_train_reshape = np.array(y_train_rnn_)

  X_val_reshape = np.reshape(X_val_rnn, (X_val_rnn.shape[0], 1, X_val_rnn.shape[1]))
  y_val_reshape = np.array(y_val_rnn)

  rnn_model = Sequential()
  rnn_model.add(SimpleRNN(units=4, input_shape=(X_train_reshape.shape[1], X_train_reshape.shape[2])))
  rnn_model.add(Dense(1, activation='sigmoid'))
  rnn_model.compile(loss='mse', optimizer='Adam')

  # Train the final model on the entire training dataset
  rnn_model.fit(X_train_reshape, y_train_reshape, epochs=100, batch_size=1, verbose=0)

  # Evaluate the model on the validation set
  loss = rnn_model.evaluate(X_val_reshape, y_val_reshape, verbose=0)
  print(f'Test loss (MSE): {loss}')

  # Make predictions on the test set
  y_pred = rnn_model.predict(X_test_reshape)

  y_pred_transform = scaler.inverse_transform(y_pred)

  predictions_rnn.append(y_pred_transform[0])

  train_rnn = pd.concat([train_rnn, test_rnn.iloc[:1]])
  test_rnn = test_rnn.iloc[1:]

  print(train_rnn.shape, test_rnn.shape)

In [ ]:
pd.DataFrame(predictions_rnn).to_csv(f'{path}/preds_rnn.csv', index=False)

In [ ]:
predictions_rnn = pd.read_csv(f'{path}/preds_rnn.csv')

In [ ]:
# Calculate evaluation metrics (RMSE, MAPE, MAE)
rmse = sqrt(mean_squared_error(y_test, predictions_rnn))
mape = mean_absolute_percentage_error(y_test, predictions_rnn)
mae = np.abs(y_test - predictions_rnn.values).mean()

print(f'Test RMSE: {rmse}')
print(f'Test MAPE: {mape}')
print(f'Test MAE: {mae}')

# Store the results
rnn_value = pd.DataFrame(data={'model': f"LSTM", 'rmse':[rmse], 'mape':[mape], 'mae':[mae]})

<a id='rnn_emd'></a><h3>3.2. RNN with CEEMDAN

In [ ]:
history = [x for x in train]
predictions_rnn_c_ceemdan = list()

train_rnn_c = train_imf[['imf_0', 'imf_1', 'imf_2', 'imf_3', 'imf_4','energy_c_scaler','energy_c_scaler_shift']]
test_rnn_c = imf.iloc[size:-1][['imf_0', 'imf_1', 'imf_2', 'imf_3', 'imf_4', 'energy_c_scaler_shift']]
print(train_rnn_c.shape, test_rnn_c.shape)

# walk-forward validation
for t in range(len(test)-1):

  X_train_rnn_c = train_rnn_c[['imf_0', 'imf_1', 'imf_2', 'imf_3', 'imf_4']]
  y_train_rnn_c = train_rnn_c['energy_c_scaler_shift']

  X_test_rnn_c = test_rnn_c[['imf_0', 'imf_1', 'imf_2', 'imf_3', 'imf_4']]
  y_test_rnn_c = test_rnn_c['energy_c_scaler_shift']

  size_train = int(len(X_train_rnn_c.values) * 0.9)
  X_val_rnn_c = X_train_rnn_c[size_train:]
  y_val_rnn_c = X_train_rnn_c[size_train:]
  X_train_rnn_c_ = X_train_rnn_c.iloc[0:size_train]
  y_train_rnn_c_ = y_train_rnn_c.iloc[0:size_train]

  X_test_reshape = np.reshape(X_test_rnn_c, (X_test_rnn_c.shape[0], 1, X_test_rnn_c.shape[1]))
  y_test_reshape = np.array(y_test_rnn_c)

  X_train_reshape = np.reshape(X_train_rnn_c_, (X_train_rnn_c_.shape[0], 1, X_train_rnn_c_.shape[1]))
  y_train_reshape = np.array(y_train_rnn_c_)

  X_val_reshape = np.reshape(X_val_rnn_c, (X_val_rnn_c.shape[0], 1, X_val_rnn_c.shape[1]))
  y_val_reshape = np.array(y_val_rnn_c)


  rnn_c_model = Sequential()
  rnn_c_model.add(SimpleRNN(units=16, input_shape=(X_train_reshape.shape[1], X_train_reshape.shape[2]), return_sequences=True))
  rnn_c_model.add(SimpleRNN(units=16))
  rnn_c_model.add(Dense(1, activation='sigmoid'))
  rnn_c_model.compile(loss='mse', optimizer='Adam')

  # Train the final model on the entire training dataset
  rnn_c_model.fit(X_train_reshape, y_train_reshape, epochs=50, batch_size=1, verbose=0)

  # Evaluate the model on the validation set
  loss = rnn_c_model.evaluate(X_val_reshape, y_val_reshape, verbose=0)
  print(f'Test loss (MSE): {loss}')

  # Make predictions on the test set
  y_pred = rnn_c_model.predict(X_test_reshape)

  y_pred_transform = scaler.inverse_transform(y_pred)

  predictions_rnn_c_ceemdan.append(y_pred_transform[0])

  train_rnn_c = pd.concat([train_rnn_c, test_rnn_c.iloc[:1]])
  test_rnn_c = test_rnn_c.iloc[1:]

  print(train_rnn_c.shape, test_rnn_c.shape)


In [ ]:
pd.DataFrame(predictions_rnn_c_ceemdan).to_csv(f'{path}/preds_rnn_c.csv', index=False)

In [ ]:
predictions_rnn_c_ceemdan = pd.read_csv(f'{path}/preds_rnn_c.csv')

In [ ]:
# Calculate evaluation metrics (RMSE, MAPE, MAE)
rmse = sqrt(mean_squared_error(y_test, predictions_rnn_c_ceemdan))
mape = mean_absolute_percentage_error(y_test, predictions_rnn_c_ceemdan)
mae = np.abs(y_test - predictions_rnn_c_ceemdan.values).mean()

print(f'Test RMSE: {rmse}')
print(f'Test MAPE: {mape}')
print(f'Test MAE: {mae}')

# Store the results
rnn_ceemdan_value = pd.DataFrame(data={'model': f"LSTM", 'rmse':[rmse], 'mape':[mape], 'mae':[mae]})

<a id='lstm'></a><h3>3.3. LSTM

In [ ]:
history = [x for x in train]
predictions_lstm = list()

train_lstm = train_imf[['energy_c_scaler','energy_c_scaler_shift']]
test_lstm = imf.iloc[size:-1][['energy_c_scaler', 'energy_c_scaler_shift']]
print(train_lstm.shape, test_lstm.shape)

# walk-forward validation
for t in range(len(test)-1):

  X_train_lstm = train_lstm[['energy_c_scaler']]
  y_train_lstm = train_lstm['energy_c_scaler_shift']

  X_test_lstm = test_lstm[['energy_c_scaler']]
  y_test_lstm = test_lstm['energy_c_scaler_shift']

  size_train = int(len(X_train_lstm.values) * 0.9)
  X_val_lstm = X_train_lstm[size_train:]
  y_val_lstm = X_train_lstm[size_train:]
  X_train_lstm_ = X_train_lstm.iloc[0:size_train]
  y_train_lstm_ = y_train_lstm.iloc[0:size_train]

  X_test_reshape = np.reshape(X_test_lstm, (X_test_lstm.shape[0], 1, X_test_lstm.shape[1]))
  y_test_reshape = np.array(y_test_lstm)

  X_train_reshape = np.reshape(X_train_lstm_, (X_train_lstm_.shape[0], 1, X_train_lstm_.shape[1]))
  y_train_reshape = np.array(y_train_lstm_)

  X_val_reshape = np.reshape(X_val_lstm, (X_val_lstm.shape[0], 1, X_val_lstm.shape[1]))
  y_val_reshape = np.array(y_val_lstm)

  lstm_model = Sequential()
  lstm_model.add(LSTM(units=4, input_shape=(X_train_reshape.shape[1], X_train_reshape.shape[2])))
  lstm_model.add(Dense(1, activation='sigmoid'))
  lstm_model.compile(loss='mse', optimizer='Adam')

  # Train the final model on the entire training dataset
  lstm_model.fit(X_train_reshape, y_train_reshape, epochs=100, batch_size=1, verbose=0)

  # Evaluate the model on the validation set
  loss = lstm_model.evaluate(X_val_reshape, y_val_reshape, verbose=0)
  print(f'Test loss (MSE): {loss}')

  # Make predictions on the test set
  y_pred = lstm_model.predict(X_test_reshape)

  y_pred_transform = scaler.inverse_transform(y_pred)

  predictions_lstm.append(y_pred_transform[0])

  train_lstm = pd.concat([train_lstm, test_lstm.iloc[:1]])
  test_lstm = test_lstm.iloc[1:]

  print(train_lstm.shape, test_lstm.shape)


In [ ]:
pd.DataFrame(predictions_lstm).to_csv(f'{path}/preds_lstm.csv', index=False)

In [ ]:
predictions_lstm = pd.read_csv(f'{path}/preds_lstm.csv')

In [ ]:
# Calculate evaluation metrics (RMSE, MAPE, MAE)
rmse = sqrt(mean_squared_error(y_test, predictions_lstm))
mape = mean_absolute_percentage_error(y_test, predictions_lstm - 1)
mae = np.abs(y_test - predictions_lstm.values).mean()

print(f'Test RMSE: {rmse}')
print(f'Test MAPE: {mape}')
print(f'Test MAE: {mae}')

# Store the results
lstm_value = pd.DataFrame(data={'model': f"LSTM", 'rmse':[rmse], 'mape':[mape], 'mae':[mae]})

<a id='lstm_emd'></a><h3>3.3. LSTM with CEEMDAN

In [ ]:
history = [x for x in train]
predictions_lstm_ceemdan = list()

train_lstm_ceemdan = train_imf[['imf_0', 'imf_1', 'imf_2', 'imf_3', 'imf_4','energy_c_scaler','energy_c_scaler_shift']]
test_lstm_ceemdan = imf.iloc[size:-1][['imf_0', 'imf_1', 'imf_2', 'imf_3', 'imf_4', 'energy_c_scaler_shift']]
print(train_lstm_ceemdan.shape, test_lstm_ceemdan.shape)

# walk-forward validation

for t in range(len(test)-1):

  X_train_lstm_ceemdan = train_lstm_ceemdan[['imf_0', 'imf_1', 'imf_2', 'imf_3', 'imf_4']]
  y_train_lstm_ceemdan = train_lstm_ceemdan['energy_c_scaler_shift']

  X_test_lstm_ceemdan = test_lstm_ceemdan[['imf_0', 'imf_1', 'imf_2', 'imf_3', 'imf_4']]
  y_test_lstm_ceemdan = test_lstm_ceemdan['energy_c_scaler_shift']

  size_train = int(len(X_train_lstm_ceemdan.values) * 0.9)
  X_val_lstm_ceemdan = X_train_lstm_ceemdan[size_train:]
  y_val_lstm_ceemdan = X_train_lstm_ceemdan[size_train:]
  X_train_lstm_ceemdan_ = X_train_lstm_ceemdan.iloc[0:size_train]
  y_train_lstm_ceemdan_ = y_train_lstm_ceemdan.iloc[0:size_train]

  X_test_reshape = np.reshape(X_test_lstm_ceemdan, (X_test_lstm_ceemdan.shape[0], 1, X_test_lstm_ceemdan.shape[1]))
  y_test_reshape = np.array(y_test_lstm_ceemdan)

  X_train_reshape = np.reshape(X_train_lstm_ceemdan_, (X_train_lstm_ceemdan_.shape[0], 1, X_train_lstm_ceemdan_.shape[1]))
  y_train_reshape = np.array(y_train_lstm_ceemdan_)

  X_val_reshape = np.reshape(X_val_lstm_ceemdan, (X_val_lstm_ceemdan.shape[0], 1, X_val_lstm_ceemdan.shape[1]))
  y_val_reshape = np.array(y_val_lstm_ceemdan)

  lstm_ceemdan_model = Sequential()
  lstm_ceemdan_model.add(LSTM(units=16, input_shape=(X_train_reshape.shape[1], X_train_reshape.shape[2]), return_sequences=True))
  lstm_ceemdan_model.add(LSTM(units=16))
  lstm_ceemdan_model.add(Dense(1, activation='sigmoid'))
  lstm_ceemdan_model.compile(loss='mse', optimizer='Adam')

  # Train the final model on the entire training dataset
  lstm_ceemdan_model.fit(X_train_reshape, y_train_reshape, epochs=50, batch_size=1, verbose=0)

  # Evaluate the model on the validation set
  loss = lstm_ceemdan_model.evaluate(X_val_reshape, y_val_reshape, verbose=0)
  print(f'Test loss (MSE): {loss}')

  # Make predictions on the test set
  y_pred = lstm_ceemdan_model.predict(X_test_reshape)

  y_pred_transform = scaler.inverse_transform(y_pred)

  predictions_lstm_ceemdan.append(y_pred_transform[0])

  train_lstm_ceemdan = pd.concat([train_lstm_ceemdan, test_lstm_ceemdan.iloc[:1]])
  test_lstm_ceemdan = test_lstm_ceemdan.iloc[1:]

  print(train_lstm_ceemdan.shape, test_lstm_ceemdan.shape)

In [ ]:
pd.DataFrame(predictions_lstm_ceemdan).to_csv(f'{path}/preds_lstm_ceemdan.csv', index=False)

In [ ]:
predictions_lstm_ceemdan = pd.read_csv(f'{path}/preds_lstm_ceemdan.csv')

In [ ]:
# Calculate evaluation metrics (RMSE, MAPE, MAE)
rmse = sqrt(mean_squared_error(y_test, predictions_lstm_ceemdan))
mape = mean_absolute_percentage_error(y_test, predictions_lstm_ceemdan - 1)
mae = np.abs(y_test - predictions_lstm_ceemdan.values).mean()

print(f'Test RMSE: {rmse}')
print(f'Test MAPE: {mape}')
print(f'Test MAE: {mae}')

# Store the results
lstm_ceemdan_value = pd.DataFrame(data={'model': f"LSTM", 'rmse':[rmse], 'mape':[mape], 'mae':[mae]})

<a id='n_prophet'></a><h3>3.4. NeuralProphet

In [ ]:
df_neuralprophet = df[['date', 'energy_c_scaler', 'energy_c_scaler_shift', 'energy_consumption', 'energy_consumption_shift']]
df_neuralprophet.columns = ['ds', 'y', 'y_shift', 'energy_consumption', 'energy_consumption_shift']

X = df_neuralprophet.values
size = int(len(X) * 0.9)
train_neuralprophet, test_neuralprophet = df_neuralprophet.iloc[0:size], df_neuralprophet.iloc[size:]

train_neuralprophet = train_neuralprophet[['ds', 'y']]
test_neuralprophet = test_neuralprophet.dropna()[['ds', 'y']]

In [ ]:
history = [x for x in train]
predictions_neuralprophet = pd.DataFrame()

train_neuralprophe_pred = train_neuralprophet[:size]
test_neuralprophet_pred = test_neuralprophet.copy()

print(train_neuralprophe_pred.shape, test_neuralprophet_pred.shape)

# walk-forward validation

for t in range(len(test_neuralprophet)):

  size_val = int(len(train_neuralprophe_pred.values) * 0.9)
  train_neuralprophe_pred_ = train_neuralprophe_pred[:size_val]
  val_neuralprophe_pred = train_neuralprophe_pred[size_val:]

  model_neuralprophet = NeuralProphet()
  model_neuralprophet.fit(train_neuralprophe_pred_, freq="M", metrics='mse',
                          validation_df=val_neuralprophe_pred)

  y_pred = model_neuralprophet.predict(test_neuralprophet_pred[:1])
  predictions_neuralprophet = pd.concat([predictions_neuralprophet, y_pred])

  train_neuralprophe_pred = pd.concat([train_neuralprophe_pred, test_neuralprophet_pred.iloc[:1]])
  test_neuralprophet_pred = test_neuralprophet_pred.iloc[1:]

  print(train_neuralprophe_pred.shape, test_neuralprophet_pred.shape)

In [ ]:
pd.DataFrame(predictions_neuralprophet).to_csv(f'{path}/preds_neuralprophet.csv', index=False)

In [ ]:
predictions_neuralprophet_transform = scaler.inverse_transform(np.array(predictions_neuralprophet['yhat1']).reshape(-1, 1))

# Calculate evaluation metrics (RMSE, MAPE, MAE)
rmse = sqrt(mean_squared_error(y_test, predictions_neuralprophet_transform))
mape = mean_absolute_percentage_error(y_test, predictions_neuralprophet_transform - 1)
mae = np.abs(y_test - predictions_neuralprophet_transform).mean()

print(f'Test RMSE: {rmse}')
print(f'Test MAPE: {mape}')
print(f'Test MAE: {mae}')

# Store the results
lstm_ceemdan_value = pd.DataFrame(data={'model': f"LSTM", 'rmse':[rmse], 'mape':[mape], 'mae':[mae]})

<a id='transformer'></a><h3>3.5. Transformer

In [ ]:
def get_datasets(num_train, num_test, num_valid, df_transformer):

  border1s = [0, num_train - context_length, len(df_transformer) - num_test - context_length,]
  border2s = [num_train, num_train + num_valid, len(df_transformer)]

  train_start_index = border1s[0]
  train_end_index = border2s[0]
  valid_start_index = border1s[1]
  valid_end_index = border2s[1]
  test_start_index = border1s[2]
  test_end_index = border2s[2]

  train_data = select_by_index(
      df_transformer,
      id_columns=id_columns,
      start_index=train_start_index,
      end_index=train_end_index)
  valid_data = select_by_index(
      df_transformer,
      id_columns=id_columns,
      start_index=valid_start_index,
      end_index=valid_end_index)
  test_data = select_by_index(
      df_transformer,
      id_columns=id_columns,
      start_index=test_start_index,
      end_index=test_end_index)

  time_series_preprocessor = TimeSeriesPreprocessor(
      timestamp_column=timestamp_column,
      id_columns=id_columns,
      input_columns=forecast_columns,
      output_columns=forecast_columns,
      scaling=False)
  time_series_preprocessor = time_series_preprocessor.train(train_data)

  train_dataset = ForecastDFDataset(
      time_series_preprocessor.preprocess(train_data),
      id_columns=id_columns,
      timestamp_column="date",
      target_columns=forecast_columns,
      context_length=context_length,
      prediction_length=forecast_horizon)
  valid_dataset = ForecastDFDataset(
      time_series_preprocessor.preprocess(valid_data),
      id_columns=id_columns,
      timestamp_column="date",
      target_columns=forecast_columns,
      context_length=context_length,
      prediction_length=forecast_horizon)
  test_dataset = ForecastDFDataset(
      time_series_preprocessor.preprocess(test_data),
      id_columns=id_columns,
      timestamp_column="date",
      target_columns=forecast_columns,
      context_length=context_length,
      prediction_length=forecast_horizon,)

  return train_dataset, valid_dataset, test_dataset

In [ ]:
timestamp_column = "date"
id_columns = []

df_transformer = df[['date', 'energy_c_scaler']]

num_input = len(df_transformer.columns) - 1
context_length = 2
forecast_columns = list(df_transformer.columns[1:])
forecast_horizon = 1
num_workers = 4
batch_size = 64

In [ ]:
num_train = int(len(df_transformer) * 0.81)
num_test = int(len(df_transformer) * 0.1)
num_valid = len(df_transformer) - num_train - num_test

predictions_transformer = pd.DataFrame()

for i in range(int(len(df_transformer) * 0.1)):
  train_dataset, valid_dataset, test_dataset = get_datasets(num_train, num_test, num_valid, df_transformer)

  finetune_forecast_model = PatchTSTForPrediction.from_pretrained(
    pretrained_model_name_or_path='ibm-research/patchtst-etth1-pretrain',
    config = PatchTSTConfig(
      context_length=2,
      prediction_length=forecast_horizon,
      loss="mse"))

  finetune_forecast_args = TrainingArguments(
      output_dir=f"{path}/output/",
      overwrite_output_dir=True,
      learning_rate=0.0001,
      num_train_epochs=20,
      do_eval=True,
      eval_strategy="steps",
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      dataloader_num_workers=num_workers,
      report_to="tensorboard",
      save_strategy="steps",
      save_steps=100,
      logging_strategy="steps",
      logging_steps=100,
      save_total_limit=2,
      logging_dir=f"{path}/logs/",  # Make sure to specify a logging directory
      load_best_model_at_end=True,  # Load the best model when training ends
      metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
      greater_is_better=False,  # For loss
      label_names=["future_values"])

  early_stopping_callback = EarlyStoppingCallback(
      early_stopping_patience=5,  # Number of epochs with no improvement after which to stop
      early_stopping_threshold=0.01)  # Minimum improvement required to consider as improvement

  finetune_forecast_trainer = Trainer(
      model=finetune_forecast_model,
      args=finetune_forecast_args,
      train_dataset=train_dataset,
      eval_dataset=valid_dataset,
      callbacks=[early_stopping_callback])

  for param in finetune_forecast_trainer.model.model.parameters():
      param.requires_grad = False

  finetune_forecast_trainer.train()
  result = finetune_forecast_trainer.evaluate(valid_dataset)

  test_result = finetune_forecast_trainer.evaluate(test_dataset)
  print(test_result)

  output = finetune_forecast_trainer.predict(test_dataset)

  try:
    y_pred_transform = output.predictions[1]
  except:
    y_pred_transform = output.predictions[0]
  display(y_pred_transform.shape)
  # print(y_pred_transform)

  predictions_transformer = pd.concat([predictions_transformer, pd.DataFrame(y_pred_transform.reshape(-1, 1)).T])
  print(predictions_transformer.shape)

  num_train = num_train + 1
  num_test = num_test - 1

In [ ]:
predictions_transformer_transform = np.array(predictions_transformer.iloc[:-1, 0]).reshape(-1, 1)
predictions_transformer_transform = scaler.inverse_transform(predictions_transformer_transform)
display(predictions_transformer_transform.shape)

pd.DataFrame(predictions_transformer).to_csv(f'{path}/preds_transformer.csv', index=False)

In [ ]:
# Calculate evaluation metrics (RMSE, MAPE, MAE)
rmse = sqrt(mean_squared_error(y_test, predictions_transformer_transform))
mape = mean_absolute_percentage_error(y_test, predictions_transformer_transform)
mae = np.abs(y_test - predictions_transformer_transform).mean()

print(f'Test RMSE: {rmse}')
print(f'Test MAPE: {mape}')
print(f'Test MAE: {mae}')

# Store the results
transformer_value = pd.DataFrame(data={'model': f"PatchTST Transformer", 'rmse':[rmse], 'mape':[mape], 'mae':[mae]})

In [ ]:
transformer_value

<a id='=transformer_emd'></a><h3>3.6. Transformer with CEEMDAN</h3>

In [ ]:
timestamp_column = "date"
id_columns = []

df_transformer_c = imf[['date', 'imf_0', 'imf_1', 'imf_2', 'imf_3', 'imf_4']]

num_input = len(df_transformer_c.columns) - 1
context_length = 2
forecast_columns = list(df_transformer_c.columns[1:])
forecast_horizon = 1
num_workers = 4
batch_size = 64

In [ ]:
num_train = int(len(df_transformer_c) * 0.81)
num_test = int(len(df_transformer_c) * 0.1)
num_valid = len(df_transformer_c) - num_train - num_test

predictions_transformer_c = pd.DataFrame()

for i in range(int(len(df_transformer_c) * 0.1)):
  train_dataset, valid_dataset, test_dataset = get_datasets(num_train, num_test, num_valid, df_transformer_c)

  finetune_forecast_model = PatchTSTForPrediction.from_pretrained(
    pretrained_model_name_or_path='ibm-research/patchtst-etth1-pretrain',
    config = PatchTSTConfig(
      num_input_channels=num_input,
      context_length=context_length,
      prediction_length=forecast_horizon,
      loss="mse"))

  finetune_forecast_args = TrainingArguments(
      output_dir=f"{path}/output/",
      overwrite_output_dir=True,
      learning_rate=0.0001,
      num_train_epochs=20,
      do_eval=True,
      eval_strategy="steps",
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      dataloader_num_workers=num_workers,
      report_to="tensorboard",
      save_strategy="steps",
      save_steps=100,
      logging_strategy="steps",
      logging_steps=100,
      save_total_limit=2,
      logging_dir=f"{path}/logs/",  # Make sure to specify a logging directory
      load_best_model_at_end=True,  # Load the best model when training ends
      metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
      greater_is_better=False,  # For loss
      label_names=["future_values"])

  early_stopping_callback = EarlyStoppingCallback(
      early_stopping_patience=5,  # Number of epochs with no improvement after which to stop
      early_stopping_threshold=0.01)  # Minimum improvement required to consider as improvement

  finetune_forecast_trainer = Trainer(
      model=finetune_forecast_model,
      args=finetune_forecast_args,
      train_dataset=train_dataset,
      eval_dataset=valid_dataset,
      callbacks=[early_stopping_callback])

  for param in finetune_forecast_trainer.model.model.parameters():
      param.requires_grad = False

  finetune_forecast_trainer.train()
  result_c = finetune_forecast_trainer.evaluate(valid_dataset)

  test_result_c = finetune_forecast_trainer.evaluate(test_dataset)
  print(test_result_c)

  output_c = finetune_forecast_trainer.predict(test_dataset)

  try:
    y_pred_transform_c = pd.DataFrame(output_c.predictions[1][0])
  except:
    y_pred_transform_c = pd.DataFrame(output_c.predictions[0][0])
  display(y_pred_transform_c.shape)

  predictions_transformer_c = pd.concat([predictions_transformer_c, y_pred_transform_c])
  print(predictions_transformer_c.shape)

  num_train = num_train + 1
  num_test = num_test - 1

In [ ]:
predictions_transformer_c.columns = ['imf_0', 'imf_1', 'imf_2', 'imf_3', 'imf_4']
predictions_transformer_c['pred'] = predictions_transformer_c.sum(axis=1)

In [ ]:
predictions_transformer_c_transform = scaler.inverse_transform(np.array(predictions_transformer_c['pred']).reshape(-1, 1))
display(predictions_transformer_c_transform.shape)

In [ ]:
pd.DataFrame(predictions_transformer_c_transform).to_csv(f'{path}/preds_transformer_c.csv', index=False)

In [ ]:
# Calculate evaluation metrics (RMSE, MAPE, MAE)
rmse = sqrt(mean_squared_error(y_test, predictions_transformer_c_transform))
mape = mean_absolute_percentage_error(y_test, predictions_transformer_c_transform)
mae = np.abs(y_test - predictions_transformer_c_transform).mean()

print(f'Test RMSE: {rmse}')
print(f'Test MAPE: {mape}')
print(f'Test MAE: {mae}')

# Store the results
transformer_c_value = pd.DataFrame(data={'model': f"PatchTST Transformer", 'rmse':[rmse], 'mape':[mape], 'mae':[mae]})

In [ ]:
predictions_arma = pd.read_csv(f'{path}/preds_arma.csv')
predictions_s_arma = pd.read_csv(f'{path}/preds_s_arma.csv')
predictions_sarima = pd.read_csv(f'{path}/preds_sarima.csv')
predictions_prophet = pd.read_csv(f'{path}/preds_prophet.csv')
predictions_rnn = pd.read_csv(f'{path}/preds_rnn.csv')
predictions_rnn_c_ceemdan = pd.read_csv(f'{path}/preds_rnn_c.csv')
predictions_lstm = pd.read_csv(f'{path}/preds_lstm.csv')
predictions_lstm_ceemdan = pd.read_csv(f'{path}/preds_lstm_ceemdan.csv')
predictions_neuralprophet = pd.read_csv(f'{path}/preds_neuralprophet.csv')
predictions_transformer = pd.read_csv(f'{path}/preds_transformer.csv')
predictions_transformer_c  = pd.read_csv(f'{path}/preds_transformer_c.csv')

In [ ]:
predictions_neuralprophet = pd.DataFrame(scaler.inverse_transform(np.array(predictions_neuralprophet['yhat1']).reshape(-1, 1)))
predictions_transformer = pd.DataFrame(scaler.inverse_transform(np.array(predictions_transformer.iloc[:-1, 0]).reshape(-1, 1)))

In [ ]:
preds = pd.concat([test[['energy_consumption']].reset_index(), predictions_arma, predictions_s_arma, predictions_sarima, predictions_prophet,
           predictions_rnn, predictions_rnn_c_ceemdan, predictions_lstm, predictions_lstm_ceemdan,
           predictions_neuralprophet, predictions_transformer, predictions_transformer_c], axis=1)

In [ ]:
preds = preds.set_index('date')

In [ ]:
preds.columns = ['test', 'ARMA', 'Seasonal ARMA', 'SARIMA', 'Prophet', 'RNN',
                 'RNN with CEEMDAN', 'LSTM', 'LSTM with CEEMDAN', 'Neural Prophet',
                 'Transformer', 'Transformer with CEEMDAN']

In [ ]:
models = ['ARMA', 'Seasonal ARMA', 'SARIMA', 'Prophet', 'RNN',
          'RNN with CEEMDAN', 'LSTM', 'LSTM with CEEMDAN', 'Neural Prophet',
          'Transformer', 'Transformer with CEEMDAN']

results = []
for model in models:
  rmse = sqrt(mean_squared_error(preds['test'], preds[model]))
  mape = mean_absolute_percentage_error(preds['test'], preds[model])
  mae = np.abs(preds['test'] - preds[model]).mean()

  results.append({'Model': model, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape})

results_df = pd.DataFrame(results)
display(results_df)